## Imports

In [27]:
from datasets import Dataset
from typing import List
from keras.utils.np_utils import to_categorical

2023-02-10 15:25:56.198110: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 15:25:56.709664: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-10 15:25:56.709759: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-10 15:25:56.777045: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-10 15:25:57.974492: W tensorflow/stream_executor/platform/de

## Constantes

In [3]:
dir = "../"
dir_data = "data/"
dir_data_rotulacao = "rotulacao/result_rotulacao/"
dir_dataset_alvo = "datasets/dataset_facebook/"

arquivo_result = "facebook_200-03-02-2023.json" ## Verificar caso a caso
dataset_alvo = "dataset_concatenate_facebook.json" ## Verificar caso a caso

## Lendo Resultados

In [4]:
resultado_doccano = Dataset.from_json(f"{dir}{dir_data}{dir_data_rotulacao}{arquivo_result}")

Using custom data configuration default-ca16ff8e041f930c
Found cached dataset json (/home/thiago/.cache/huggingface/datasets/json/default-ca16ff8e041f930c/0.0.0)


## Lendo Dataset

In [5]:
dataset = Dataset.from_json(f"{dir}{dir_data}{dir_dataset_alvo}{dataset_alvo}")

Using custom data configuration default-919784d60fd56bf4
Found cached dataset json (/home/thiago/.cache/huggingface/datasets/json/default-919784d60fd56bf4/0.0.0)


## Prepara Dados Resultados

In [12]:
resultado_doccano

Dataset({
    features: ['id', 'text', 'index', 'label', 'Comments'],
    num_rows: 200
})

### Remover Colunas Desnecessárias

In [19]:
resultado_doccano = resultado_doccano.remove_columns(["id", "text", "Comments"])

### Arrumar Colunas de Label

In [15]:
labels: List[str] = list([])
for lista_label in resultado_doccano["label"]:
    labels.extend(lista_label)

In [24]:
resultado_doccano = resultado_doccano.add_column("label_str", labels)

In [26]:
set(resultado_doccano["label_str"])

{'Negativo', 'Neutro', 'Positivo'}

In [28]:
def arruma_label(data):
    valor = 0
    if data["label_str"] == "Neutro":
        valor = 1
    elif data["label_str"] == "Positivo":
        valor = 2
    data["label"] = to_categorical(valor, num_classes=3)
    return data
resultado_doccano = resultado_doccano.map(arruma_label)

  0%|          | 0/200 [00:00<?, ?ex/s]

In [32]:
resultado_doccano.features

{'index': Value(dtype='int64', id=None),
 'label': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
 'label_str': Value(dtype='string', id=None)}

### Prepara Dados Dataset

In [40]:
if "label" not in dataset.features:
    dataset = dataset.add_column("label", [None] * len(dataset))

In [38]:
if "label_str" not in dataset.features:
    dataset = dataset.add_column("label_str", [""] * len(dataset))

In [44]:
dataset.features

{'created_time': Value(dtype='string', id=None),
 'message': Value(dtype='string', id=None),
 'permalink_url': Value(dtype='string', id=None),
 'id_post': Value(dtype='string', id=None),
 'dataset_origem': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None),
 'label_str': Value(dtype='string', id=None),
 'label': Value(dtype='null', id=None)}

In [43]:
dataset[0]

{'created_time': '2021-07-08T22:29:49+0000',
 'message': 'É um whisky pra quem não entende nada de whisky, me bloquearam no Instagram por falar a verdade sobre whiskys, empresa piada, sei que logo vai apagar meu comentário aqui também. Pra todos que verem esse meu comentário, saiba que não recomendo a casa das bebidas, a Amazon tem muito mais variedade e preço!! Fui',
 'permalink_url': 'https://www.facebook.com/120086268098178/posts/4083651658408266/?comment_id=4163733760400055',
 'id_post': '120086268098178_4083651658408266',
 'dataset_origem': 'extracao_comentarios_pagina_casadabebida.csv',
 'index': 0,
 'label_str': '',
 'label': None}